# Cancer Type Classifier
- BME 230A class project winter 2019
- Andrew E. Davidson
- [aedavids@ucsc.edu](mailto:aedavids@edu?subject=SimpleModel.ipynb)

classify cancer type from gene expression

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

import matplotlib.pyplot as plt
import numpy as np
import os 
import pandas as pd

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

import sys

# fix random seed for reproducibility
theMeaningOfLife = 42

import keras
if "2.1.6" != keras.__version__ :
    emsg = "ERROR keras version {} != 2.1.6, new version can not save and restore models".format(keras.__version__)
    raise ValueError(emsg)

# add path to our local modules
# assume they are in the same directory we launched the juypter server in
# /home/ubuntu/BME-230a
!pwd
localModuleDir = "."
sys.path.append(localModuleDir)
from loadData import loadTumorNormalData

In [ ]:
rootDir = "/bme-230a-ebs"
sourceDataFilePath = "{}/data/tcga_target_gtex.h5".format(rootDir)
print(sourceDataFilePath)
if not os.path.isfile(sourceDataFilePath) :
    emsg = "ERROR: {} not found".format(sourceDataFilePath)
    print(emsg)
    print("change rootDir")
    sys.stdout.flush() # force error message to print
    raise ValueError(emsg)

In [ ]:
%%time
# Load training set
XDF = pd.read_hdf(sourceDataFilePath, "expression")
print("XDF.shape:{}".format(XDF.shape))

yDF = pd.read_hdf(sourceDataFilePath, "labels")
print("yDF.shape:{} type(yDF):".format(yDF.shape)) 

In [ ]:
XDF.head(1)

In [ ]:
yDF.head(1)

In [ ]:
np.array_equal(pd.unique( yDF.loc[:,"disease"] ), pd.unique( yDF.loc[:,"category"] ))

In [ ]:
diseaseClasses = pd.unique( yDF.loc[:,"disease"] )
diseaseK = len(diseaseClasses)
print("diseaseK: {}, type(diseaseClasses): {}".format(diseaseK, type(diseaseClasses)))
print(diseaseClasses)

In [ ]:
def plotCategoryCounts(df, colNameStr, ):
    '''
    TODO: make this more generic
    argument 
        df: a pandas data frame 
        colNameStr: 
        
    returns 
        (fig, ax)
    '''
    # countDF is series
    countDF = df[colNameStr].value_counts()
    fig, ax = plt.subplots()
    fig.set_size_inches(5,18)
    ax.barh(countDF.index, countDF)    
    
    return (fig, ax)

fig, ax = plotCategoryCounts(yDF, "disease")
ax.set_title("disease counts")

In [ ]:
# Convert disease  into numerical values 
encoder = LabelEncoder()
yDF["disease_value"] = pd.Series(encoder.fit_transform(yDF["disease"]), index=yDF.index)
yDF[["disease","disease_value"]].head(3)

In [ ]:
#%%time
# Split into stratified training and test sets based on classes (i.e. tissue type) so that we have equal
# proportions of each tissue type in the train and test sets
from sklearn.model_selection import StratifiedShuffleSplit
# split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=theMeaningOfLife)
# for train_index, test_index in split.split(XDF.values, yDF["disease_value"]):
#     XTrainDF = XDF[train_index]
#     XTestDF  = XDF[test_index]
#     yTrainSeries = yDF["disease_value"][train_index]
#     yTestSeries  = yDF["disease_value"][test_index]


# split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=theMeaningOfLife)
# for train_index, test_index in split.split(X.values, Y["tumor_normal_value"]):
#     X_train, X_test = X.values[train_index], X.values[test_index]
#     y_train, y_test = Y["tumor_normal_value"][train_index], \
#                         Y["tumor_normal_value"][test_index]

# Split into stratified training and test sets based on classes (i.e. tissue type) so that we have equal
# proportions of each disease type in the train and test sets
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=theMeaningOfLife)
for train_index, test_index in split.split(XDF.values, yDF["disease_value"]):
    XTrainNumpy, XTestNumpy = XDF.values[train_index], XDF.values[test_index]
    yTrainSeries, yTestSeries = yDF["disease_value"][train_index], \
                        yDF["disease_value"][test_index]
    
print("XTrainNumpy.shape: {} XTestNumpy.shape: {}".format(XTrainNumpy.shape, XTestNumpy.shape))
print(type(XTrainNumpy))
print("yTrainSeries.shape: {} yTestSeries.shape: {}".format(yTrainSeries.shape, yTestSeries.shape))
print(type(yTestSeries))